# import libaries

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from imblearn.pipeline import Pipeline 
from imblearn.combine import SMOTETomek, SMOTEENN 
from imblearn.under_sampling import TomekLinks, EditedNearestNeighbours
from imblearn.over_sampling import SMOTEN
from sklearn.preprocessing import StandardScaler
from collections import Counter

# Import the dataset 

In [2]:
df=pd.read_csv('../heart_disease/heart_disease.csv')
#df.isnull().sum() #check for null values 
#df.columns

In [3]:
model = DecisionTreeClassifier()

# Combined: SmoteTomek

In [8]:
X = df.drop (labels = ['HeartDiseaseorAttack'], axis =1)
y = df['HeartDiseaseorAttack']

smt= SMOTEN(sampling_strategy=0.3, n_jobs=-1)
tomek= TomekLinks(sampling_strategy='majority', n_jobs=-1)

resample = SMOTETomek(tomek=TomekLinks(sampling_strategy='majority'))

pipeline = Pipeline([('StandardScaler', StandardScaler()), ('r', resample), ('model', model)])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

pipeline.fit(X_train, y_train) 

y_hat = pipeline.predict(X_test)
print(classification_report_imbalanced(y_test, y_hat))

                   pre       rec       spe        f1       geo       iba       sup

        0.0       0.92      0.91      0.29      0.92      0.51      0.28     68840
        1.0       0.25      0.29      0.91      0.26      0.51      0.24      7264

avg / total       0.86      0.85      0.35      0.85      0.51      0.27     76104



# Combined: SMOTEENN

In [9]:
X = df.drop (labels = ['HeartDiseaseorAttack'], axis =1)
y = df['HeartDiseaseorAttack']

smt=SMOTEN(sampling_strategy=0.3, n_jobs=-1)
enn1=EditedNearestNeighbours(sampling_strategy= 'majority')

resample = SMOTEENN(random_state=42, n_jobs=-1, enn=enn1, smote=smt)

pipeline = Pipeline([('StandardScaler', StandardScaler()), ('r', resample), ('model', model)])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

pipeline.fit(X_train, y_train) 

y_hat = pipeline.predict(X_test)
print(classification_report_imbalanced(y_test, y_hat))

                   pre       rec       spe        f1       geo       iba       sup

        0.0       0.94      0.82      0.52      0.88      0.65      0.44     68840
        1.0       0.23      0.52      0.82      0.32      0.65      0.42      7264

avg / total       0.87      0.79      0.55      0.82      0.65      0.44     76104



# without pipeline (for testing)

In [5]:
from collections import Counter

print(f'Original dataset shape {Counter(y)}')

X_train, X_test, y_train , y_test = train_test_split(X,y,test_size=0.3, random_state=0) 

print(f'Y train {Counter(y_train)}')
print(f'Y test {Counter(y_test)}')

smt=SMOTEN(sampling_strategy=0.3, n_jobs=-1)
enn1=EditedNearestNeighbours(sampling_strategy= 'majority')

resample = SMOTEENN(random_state=42, n_jobs=-1, enn=enn1, smote=smt)

X_balanced, y_balanced = resample.fit_resample(X_train, y_train)

print(f'Y balanced {Counter(y_balanced)}')

Original dataset shape Counter({0.0: 229787, 1.0: 23893})
Y train Counter({0.0: 160947, 1.0: 16629})
Y test Counter({0.0: 68840, 1.0: 7264})
Y balanced Counter({0.0: 125829, 1.0: 48284})


In [6]:
model = DecisionTreeClassifier()

model.fit(X_balanced, y_balanced)
y_pred = model.predict(X_test)

print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

        0.0       0.94      0.83      0.49      0.88      0.64      0.42     68840
        1.0       0.23      0.49      0.83      0.32      0.64      0.39      7264

avg / total       0.87      0.80      0.52      0.83      0.64      0.42     76104

